Assumptions:
Purpose: Create embeddings for text data.
Model: Use "gpt-4o-mini" for creating embeddings.
Data: Your JSON file contains fields like title and abstract.

In [ ]:
!pip install --upgrade openai
!pip install ijson
!pip install langchain
!pip install langchain-openai
!pip install faiss-cpu
!pip install tiktoken
!pip install openai
!pip install senetence-transformers
!pip install langchain-community # install the missing package
!pip install sentence_transformers
!pip install --upgrade langchain langchain_community openai tqdm ijson sentence-transformers
!pip install transformers
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

ERROR: Could not find a version that satisfies the requirement senetence-transformers (from versions: none)
ERROR: No matching distribution found for senetence-transformers


In [ ]:
import os
from pathlib import Path
import hashlib
from tqdm import tqdm
import ijson
import time
import random
import pickle
import io

# Google Drive imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload

# Langchain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# OpenAI import
from openai import OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
json_file_path = '/content/drive/My Drive/corpus-engineering.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Google Drive setup
SCOPES = ['https://www.googleapis.com/auth/drive.file']

def get_google_drive_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('drive', 'v3', credentials=creds)

drive_service = get_google_drive_service()

# Function to save file to Google Drive
def save_to_drive(filename, content):
    file_metadata = {'name': filename}
    media = MediaIoBaseUpload(io.BytesIO(content), mimetype='application/octet-stream')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

# Function to load file from Google Drive
def load_from_drive(file_id):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    return fh.getvalue()

In [ ]:


# Set your API key directly on the openai module
#sk-ZynaDTOEaCa1wNQ9jDvBUuBsmvfBWdIbBTAeYdUPvZT3BlbkFJjwnWvgFL4RclHqa8RtndnXa-FExiCDvT5Q7OKN-agA
OpenAI.api_key = ""
client = OpenAI(api_key=OpenAI.api_key)

# Use a local embedding model
embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v2")

# Text splitter for Persian
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n', '،', '؛', '.'],  # Persian-specific separators
    chunk_size=1000,
    chunk_overlap=50
)

# Prepare document from a single JSON item
def prepare_document(item):
    text = f"شناسه: {item['id']}\nعنوان: {item['title']}\nچکیده: {item['abstract']}\nموضوع اول: {item['FirstSubject']}\nموضوع دوم: {item['SecondSubject']}"
    return text_splitter.create_documents([text])

# Create or load vector database


  # Process documents in batches
def process_documents(file_path, batch_size=1000):
    with open(file_path, 'rb') as f:
        # Get total number of items for progress bar
        parser = ijson.parse(f)
        total_items = sum(1 for _ in parser if _ == ('','start_map'))

    with open(file_path, 'rb') as f:
        items = ijson.items(f, 'item')
        batch = []
        for item in tqdm(items, total=total_items, desc="Processing documents"):
            batch.extend(prepare_document(item))
            if len(batch) >= batch_size:
                yield batch
                batch = []
        if batch:
            yield batch

def get_vector_db(file_path, force_rebuild=False):
    # Create 'embeddings' folder in the current working directory
    embeddings_folder = Path("embeddings")
    embeddings_folder.mkdir(exist_ok=True)

    file_hash = hashlib.md5(Path(file_path).read_bytes()).hexdigest()
    faiss_index_path = embeddings_folder / f"{file_hash}.faiss"
    index_metadata_path = embeddings_folder / f"{file_hash}_metadata.pkl"

    if not force_rebuild and faiss_index_path.exists() and index_metadata_path.exists():
        print(f"Loading existing FAISS index from {faiss_index_path}...")
        vector_db = FAISS.load_local(str(faiss_index_path), embeddings , allow_dangerous_deserialization=True)
        with open(index_metadata_path, 'rb') as f:
            vector_db.docstore._dict = pickle.load(f)
        return vector_db

    print("Building new FAISS index...")
    vector_db = None
    for batch in process_documents(file_path):
        if vector_db is None:
            vector_db = FAISS.from_documents(batch, embeddings)
        else:
            vector_db.add_documents(batch)

    print(f"Saving FAISS index to {faiss_index_path}...")
    vector_db.save_local(str(faiss_index_path))
    with open(index_metadata_path, 'wb') as f:
        pickle.dump(vector_db.docstore._dict, f)

    return vector_db

def generate_answer(query, contents, max_retries=5):
    prompt = f"به زبان فارسی به پرسش زیر پاسخ دهید. پاسخ را بر اساس اطلاعات داده شده ارائه دهید:\n\nپرسش: {query}\n\nاطلاعات:\n{contents}"
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt == max_retries - 1:
                raise e
            time.sleep(2 ** attempt + random.random())

def main():
    vector_db = get_vector_db(json_file_path)

    while True:
        query = input("سوال خود را به فارسی بپرسید (یا 'خروج' را برای پایان وارد کنید): ")
        if query.lower() == 'خروج':
            break

        results = vector_db.similarity_search(query, k=2)
        contents = "\n".join([doc.page_content for doc in results])

        answer = generate_answer(query, contents)
        print(f"پاسخ: {answer}\n")

if __name__ == "__main__":
    main()



Loading existing FAISS index from embeddings/7cee48aa58c44b086d4aeff192ce17b2.faiss...
سوال خود را به فارسی بپرسید (یا 'خروج' را برای پایان وارد کنید): رابطه بین فقر اقتصادی - اجتماعی روستایی و تخریب مراتع
پاسخ: پاسخ:

رابطه بین فقر اقتصادی - اجتماعی روستایی و تخریب مراتع به‌ویژه در مناطق روستایی حائز اهمیت است. در تحقیق مورد نظر که در بخش مرکزی شهرستان فارسان انجام شده است، به‌ویژه بر روی کشاورزان روستای خردمند متمرکز شده است. یافته‌های این پژوهش نشان‌دهنده آن است که بخش عمده‌ای از کشاورزان در این منطقه، مردان متاهل میانسالی هستند که به‌طور کلی با سطح سواد نسبتاً پایینی (کمتر از دیپلم) مواجه‌اند و از نظر اقتصادی در وضعیت مناسب قرار ندارند.

فقر اقتصادی - اجتماعی معمولاً باعث فشار بیشتری بر روی منابع طبیعی مانند مراتع می‌شود. در شرایط اقتصادی نامناسب، کشاورزان ممکن است به‌منظور تأمین نیازهای فوری و معیشتی خود، به بهره‌برداری بیش از حد از مراتع و زمین‌های کشاورزی روی آورند. این عمل نه تنها به تخریب منابع طبیعی منجر می‌شود، بلکه موجب کاهش کیفیت خاک و تنوع زیستی نیز خواهد شد. 

علاوه بر ا

KeyboardInterrupt: Interrupted by user